In [3]:

import pymongo
import os
import requests
import json

In [21]:
# Connect to MongoDB
client = pymongo.MongoClient(os.environ.get("MONGO_URI"))
db = client[os.environ.get("DATABASE")]
collection = db[os.environ.get("COLLECTION")]

In [67]:
from bson import ObjectId
def convert_json_to_text(json_obj):
    """
    Converts a JSON object to simple text.

    Args:
        json_obj (dict): A JSON object.

    Returns:
        str: The JSON object converted to simple text.
    """
    def default_encoder(obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

    return json.dumps(json_obj, default=default_encoder)

In [91]:

def custom_embedding(texts, model_id="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Generates embeddings for a list of texts using the Hugging Face API.

    Args:
        texts (list): A list of strings containing the text inputs.
        model_id (str, optional): The ID of the pre-trained model to use. Defaults to "sentence-transformers/all-MiniLM-L6-v2".

    Returns:
        list: A list of embeddings, where each embedding is a dense vector representing the corresponding text.
    """
    # Read the JSON file and extract the values
    texts = convert_json_to_text(texts)
    print(texts)
    print(type(texts))
    print("-----------------------")
        
    url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"

    # Assuming you have a Hugging Face access token stored securely (not recommended in production)
    headers = {"Authorization": os.environ.get("HUGGINGFACE_TOKEN")}

    # data = {"inputs": []}
    
    # for text in texts:
    #     # Convert _id to string if it exists
    #     if hasattr(text, "_id"):
    #         text["_id"] = str(text["_id"])
    #     data["inputs"].append(text)
    # print(data)
    
    response = requests.post(url, headers=headers, json=texts)
    if response.status_code == 200:
        embeddings = response.json()
        return embeddings
    else:
        print(response.json())
        print(f"Error: {response.status_code}")
        return None


In [86]:
cluster_data = collection.find()

In [94]:
# Iterate over the data and add the embedding to each document
for data in cluster_data:
    print(data)
    embedding = custom_embedding(data)
    # collection.update_one({"_id": data["_id"]}, {"$set": {"embedding": embedding}})
    print(embedding)
    break

{'_id': ObjectId('6600612b87f42350f791c9c7'), 'Unnamed: 0': 5, 'Airline Name': 'Adria Airways', 'Overall_Rating': '1', 'Review_Title': '"Had very bad experience"', 'Review Date': '17th September 2019', 'Verified': True, 'Review': '  Had very bad experience with rerouted and cancelled flights last weekend with Adria airways. Original Route was Ljubljana to Sarajevo return. Two weeks before i received an email that the flight was cancelled. Offered route change was Ljubljana to Sarajevo via Munich. Flight back changed to Sarajevo-Pristina-Ljubljana. I accepted. The first flight via Munich was ok. Two hours before the return flight I got the email that the flight was cancelled. I had to rebook via hotline and had to accept a flight with Croatian to Zagreb. I reached Ljubljana 4 h later and had to organize Transport from Zagreb to Ljubljana on my own cost. Do not book flights with Adria airways. I heard that their financial situation is very very bad.', 'Aircraft': 'CR 900', 'Type Of Trave